In [2]:
import pandas as pd
import numpy as np
import keplergl as kgl
from dksr import extract_sample_network
from geojson import LineString

In [3]:
#north, east, south, west = 33.798, -84.378, 33.763, -84.422 

In [4]:
#berlin
north, east, south, west = 52.460298,13.328991,52.515803,13.455849

In [5]:
routes = extract_sample_network(north, east, south, west, 50)

In [6]:
geo_list = pd.DataFrame(np.zeros((len(routes)),dtype=object),columns=['geo_json'])
                            
for i in range(0,len(routes)):                       
    z_list = [0] * len(routes['coordinates'][i])
    list0 = routes['coordinates'][i]
    list1 = np.insert(list0,2,z_list,axis=1)
        
    geo_list.iloc[i] = [LineString(list1.tolist())]

In [7]:
geo_list.iloc[0]

geo_json    {'type': 'LineString', 'coordinates': [[52.463...
Name: 0, dtype: object

In [8]:
#OSMnx returns values in lat, lng while KeplerGL expetcs them in lng,lat
def switch_lat_lon(coords):
    return [coords[1], coords[0], coords[2]]

In [9]:
# Apply the function to the 'coordinates' column using a lambda function
geo_list['geo_json'] = geo_list['geo_json'].apply(lambda x: {'type': x['type'], 'coordinates': [switch_lat_lon(coord) for coord in x['coordinates']]})


In [10]:
#a first test plot
map_0 = kgl.KeplerGl(height=800,data={'Scooters': geo_list})
map_0

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Scooters':                                              geo_json
0   {'type': 'LineString', 'c…

In [11]:
geo_list.iloc[0]

geo_json    {'type': 'LineString', 'coordinates': [[13.371...
Name: 0, dtype: object

In [12]:
# further transform routes data
# 1. interpolate the various LineStrings by shapely LineString Interoplate function
# 2. generate timestamps for every interpolated point on a LineString

In [36]:
from shapely.geometry import LineString
from shapely.ops import unary_union
from shapely import MultiPoint

In [19]:
test_route = geo_list.iloc[0]['geo_json']['coordinates']

In [21]:
len(test_route)

45

In [77]:
line = LineString(test_route)

In [78]:
distance_delta = 0.001

In [79]:
distances = np.arange(0, line.length, distance_delta)

In [80]:
points = MultiPoint([line.interpolate(distance) for distance in distances])

In [82]:
len(points)

TypeError: object of type 'MultiPoint' has no len()